In [1]:
import pandas as pd
import glob
import re
from const import COlS
from datetime import date, datetime
import logging


In [2]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
# Create a logger
logger = logging.getLogger(__name__)

In [3]:
logger.info('Importing file Map Owner...')
Rep_files = glob.glob("./input/*.csv")
Map_path = glob.glob("./input/Map*.xlsx")
Map_owner = pd.read_excel(Map_path[0], engine='openpyxl').astype(str)

2025-03-06 11:47:29,414 - INFO - Importing file Map Owner...


In [4]:
logger.info('Importing file Map Owner...')
Rep_files = glob.glob("./input/*.csv")
Map_path = glob.glob("./input/Map*.xlsx")
Map_owner = pd.read_excel(Map_path[0], engine='openpyxl').astype(str)
Map_owner = Map_owner.drop(columns={'Assign Date'})
Map_owner = Map_owner.rename(columns={'Loan No.': 'Loan No', 'Owner': 'OA', 'Due': 'DUE'})
logger.info('Map Owner impoerted done!!')

file_date = re.findall(r'\d+', Rep_files[0])[0]
file_time = re.findall(r'\d+', Rep_files[0])[1]
logger.info('Importing Rep file...')

Rep = pd.read_csv(Rep_files[0], usecols=COlS.use_cols, encoding='cp874').astype(str)
Rep = Rep.merge(Map_owner, on='Loan No', how='left')
logger.info('Rep file impoerted done!!')
logger.info('Start process cleanning and formatting files...')

2025-03-06 11:47:37,053 - INFO - Importing file Map Owner...
2025-03-06 11:47:41,976 - INFO - Map Owner impoerted done!!
2025-03-06 11:47:41,976 - INFO - Importing Rep file...
2025-03-06 11:47:42,912 - INFO - Rep file impoerted done!!
2025-03-06 11:47:42,912 - INFO - Start process cleanning and formatting files...


In [5]:
payment_path = glob.glob("./output/*.xlsx")
print(payment_path)


['./output\\payment_splits.xlsx']


In [6]:
def read_all_sheets(file_path):
    xls = pd.ExcelFile(file_path, engine="openpyxl")  # Load the Excel file
    all_sheets = []  # List to store individual DataFrames

    for sheet_name in xls.sheet_names:  # Iterate over all sheets
        print(f"Reading sheet: {sheet_name}")  # Debugging statement
        df = pd.read_excel(xls, sheet_name=sheet_name, dtype=str)  # Read each sheet as string
        all_sheets.append(df)

    # Combine all DataFrames into one
    if all_sheets:
        final_df = pd.concat(all_sheets, ignore_index=True)
    else:
        final_df = pd.DataFrame()  # Empty DataFrame if no sheets are found
    
    return final_df

In [7]:
payment_path = glob.glob("./output/*.xlsx")[0]


In [8]:
payment = read_all_sheets(payment_path)

Reading sheet: pay1
Reading sheet: pay2
Reading sheet: pay3
Reading sheet: pay4
Reading sheet: pay5
Reading sheet: pay6


In [9]:
payment

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
1,5000159071000091,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000,RC120250201027417057,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
2,7241070104111232,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,1471.45,RC120250131027294430,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
3,5000159071005450,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000,RC120250201027275293,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
4,7240670103101814,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,644.14,RC120250201027541218,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951407,5000159071002027,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,2282.9,RC120250226028497421,A301:RL,CFNCRC0109B,KOR,1,NaN,26
951408,5000159071001454,20250226,20250226,1,11333022:BBL(Realtime),04:Partial Early Payment,4658.68,RC120250226028436496,A301:RL,CFNCRC0109B,NaN,NaN,NaN,26
951409,5000159071017946,20250226,20250226,1,11333022:BBL(Realtime),04:Partial Early Payment,1176.45,RC120250226028441234,A301:RL,CFNCRC0109B,NaN,NaN,NaN,26
951410,5000159101090981,20250226,20250226,1,11333022:BBL(Realtime),04:Partial Early Payment,1000,RC120250226028495115,A301:RL,CFNCRC0109B,NaN,NaN,NaN,26


In [10]:
common_duplicate = Rep[Rep['Receipt Number'].isin(payment['Receipt Number'])]
# common_duplicate
common_duplicate.to_excel('duplicated_27&28.xlsx', index=False, engine='xlsxwriter')
common_duplicate.groupby('Receipt date').size()
# Rep.groupby('Receipt date').size()

Receipt date
20250228    1
dtype: int64

In [11]:
common_unique = Rep[~Rep['Receipt Number'].isin(payment['Receipt Number'])]
# common_unique.to_excel('unique_28.xlsx', index=False, engine='xlsxwriter')
common_unique.groupby('Receipt date').size()

Receipt date
20250228    263099
dtype: int64

In [12]:
Rep

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE
0,1000162080004110,20250228,20250228,1,11333022:BBL(Realtime),08:Payment for written-off Loan,1500.0,RC120250228028694427,A501:TL,CFNCRC0109B,NaN,NaN
1,1000162120000837,20250228,20250228,1,11333022:BBL(Realtime),08:Payment for written-off Loan,1500.0,RC120250228028673393,A501:TL,CFNCRC0109B,MMN,WO
2,1000163010000841,20250228,20250228,1,11333022:BBL(Realtime),08:Payment for written-off Loan,1400.0,RC120250228028794622,A501:TL,CFNCRC0109B,GMV,WO
3,1000163020001077,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,2011.0,RC120250228028624920,A501:TL,CFNCRC0109B,NaN,NaN
4,1000163020001107,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,2352.0,RC120250228028731656,A501:TL,CFNCRC0109B,PCN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
263095,9010221112000661,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,2860.0,RC120250228028785404,A501:TL,CFNCRC0109B,UPC,1
263096,9010221122000566,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,708.0,RC120250228028815252,A501:TL,CFNCRC0109B,ACT,1
263097,9010221122000744,20250228,20250228,1,11333022:BBL(Realtime),08:Payment for written-off Loan,2000.0,RC120250228028768509,A501:TL,CFNCRC0109B,GMV,WO
263098,9010221122000744,20250228,20250228,2,11333022:BBL(Realtime),08:Payment for written-off Loan,1020.0,RC120250228028798602,A501:TL,CFNCRC0109B,GMV,WO


In [13]:
payment_combine = pd.concat([payment, Rep])
payment_combine

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
1,5000159071000091,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000,RC120250201027417057,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
2,7241070104111232,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,1471.45,RC120250131027294430,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
3,5000159071005450,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000,RC120250201027275293,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
4,7240670103101814,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,644.14,RC120250201027541218,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263095,9010221112000661,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,2860.0,RC120250228028785404,A501:TL,CFNCRC0109B,UPC,1,NaN,NaN
263096,9010221122000566,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,708.0,RC120250228028815252,A501:TL,CFNCRC0109B,ACT,1,NaN,NaN
263097,9010221122000744,20250228,20250228,1,11333022:BBL(Realtime),08:Payment for written-off Loan,2000.0,RC120250228028768509,A501:TL,CFNCRC0109B,GMV,WO,NaN,NaN
263098,9010221122000744,20250228,20250228,2,11333022:BBL(Realtime),08:Payment for written-off Loan,1020.0,RC120250228028798602,A501:TL,CFNCRC0109B,GMV,WO,NaN,NaN


In [14]:
payment_combine= payment_combine.sort_values(by='Receipt date')
payment_combine.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8124
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83529
20250228    263100
dtype: int64

In [15]:
payment_combine['Payment Amt(Total payment Amt)'] = payment_combine['Payment Amt(Total payment Amt)'].astype('float64')
payment_combine

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
204407,7240870103634487,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,662.58,RC120250201027716963,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204406,7240670103031306,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,388.32,RC120250201027466785,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
204405,7240870103634468,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250131027287501,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204404,7240670103101817,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,662.55,RC120250201027373145,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87704,7240470102540793,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,899.51,RC120250228028682906,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,NaN
87705,7240470102540795,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,771.00,RC120250228028727340,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,NaN
87706,7240470102540796,20250228,20250228,1,11333026:M-pay(Realtime SFP),01:Normal Payment,552.13,RC120250228028711853,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,NaN
87612,7240470102539194,20250228,20250228,1,11333026:M-pay(Realtime SFP),01:Normal Payment,629.66,RC120250228028684233,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,NaN


In [16]:
# Drop rows where 'number' is duplicated
payment_unique = payment_combine[payment_combine.duplicated('Receipt Number', keep=False) == False]
payment_unique

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
204407,7240870103634487,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,662.58,RC120250201027716963,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204406,7240670103031306,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,388.32,RC120250201027466785,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
204405,7240870103634468,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250131027287501,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204404,7240670103101817,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,662.55,RC120250201027373145,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87704,7240470102540793,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,899.51,RC120250228028682906,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,NaN
87705,7240470102540795,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,771.00,RC120250228028727340,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,NaN
87706,7240470102540796,20250228,20250228,1,11333026:M-pay(Realtime SFP),01:Normal Payment,552.13,RC120250228028711853,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,NaN
87612,7240470102539194,20250228,20250228,1,11333026:M-pay(Realtime SFP),01:Normal Payment,629.66,RC120250228028684233,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,NaN


In [17]:
payment_res = payment_unique[payment_unique['Payment Amt(Total payment Amt)'].astype('float64') > 0]
payment_res.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8123
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83529
20250228    262647
dtype: int64

In [19]:
payment_res

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
204407,7240870103634487,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,662.58,RC120250201027716963,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204406,7240670103031306,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,388.32,RC120250201027466785,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
204405,7240870103634468,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250131027287501,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204404,7240670103101817,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,662.55,RC120250201027373145,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87704,7240470102540793,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,899.51,RC120250228028682906,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,NaN
87705,7240470102540795,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,771.00,RC120250228028727340,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,NaN
87706,7240470102540796,20250228,20250228,1,11333026:M-pay(Realtime SFP),01:Normal Payment,552.13,RC120250228028711853,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,NaN
87612,7240470102539194,20250228,20250228,1,11333026:M-pay(Realtime SFP),01:Normal Payment,629.66,RC120250228028684233,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,NaN


In [20]:
payment_final = payment_res[~payment_res['Receipt channel'].str.contains('CIMB')].replace('nan', None).sort_values(by='Receipt date')
payment_final

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
15,1000163120000064,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,4694.00,RC120250201027620545,A101:C4C,CFNCRC0109B,NaN,NaN,NaN,1
153302,7250170104548055,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,331.27,RC120250201027331674,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
1,5000159071000091,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000.00,RC120250201027417057,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
2,7241070104111232,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,1471.45,RC120250131027294430,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262841,8002060011078791,20250228,20250228,1,11333022:BBL(Realtime),04:Partial Early Payment,3000.00,RC120250228028656764,A301:RL,CFNCRC0109B,NaN,NaN,NaN,NaN
262587,8002059091056100,20250228,20250228,1,11333022:BBL(Realtime),04:Partial Early Payment,1666.80,RC120250228028626912,A301:RL,CFNCRC0109B,NaN,NaN,NaN,NaN
262842,8002060021012121,20250228,20250228,1,11333022:BBL(Realtime),01:Normal Payment,1680.00,RC120250228028763035,A301:RL,CFNCRC0109B,NaN,NaN,NaN,NaN
262848,8002060021014159,20250228,20250228,1,11333022:BBL(Realtime),04:Partial Early Payment,1480.89,RC120250228028717359,A301:RL,CFNCRC0109B,NaN,NaN,NaN,NaN


In [21]:
payment_final.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8123
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83529
20250228    262646
dtype: int64

In [38]:
import pandas as pd

def auto_split_payment_data(payment_final, range_size=5):
    # Convert the 'date' column to datetime if it's not already in that format
    payment_final['datechunk'] = pd.to_datetime(payment_final['Receipt date'])
    
    # Get the day of the month from the 'date' column
    payment_final['datechunk'] = payment_final['datechunk'].dt.day
    
    # Get the maximum day value in the data to determine the number of ranges
    max_day = payment_final['datechunk'].max()

    # Create a dictionary to store the splits
    payment_splits = {}

    # Loop through the days in the specified range and create splits
    for start_day in range(1, max_day + 1, range_size):
        end_day = min(start_day + range_size - 1, max_day)
        pay_key = f'pay{(start_day - 1) // range_size + 1}'
        
        # Filter payment data based on the current day range
        payment_splits[pay_key] = payment_final[(payment_final['datechunk'] >= start_day) & 
                                                (payment_final['datechunk'] <= end_day)]

    return payment_splits




In [39]:
def save_payment_splits_to_excel(payment_final, file_name, range_size=5):
    # Split the data dynamically
    payment_splits = auto_split_payment_data(payment_final, range_size)
    # output_dir='./output/'
    # os.makedirs(output_dir, exist_ok=True)
    
    # Save the splits to an Excel file
    with pd.ExcelWriter(f'./output/{file_name}', engine='xlsxwriter') as writer:
        # Loop through the payment splits and save each as a sheet
        for pay_key, pay_data in payment_splits.items():
            print('pay_data',len(pay_data))
            # Save the split data to an Excel sheet with the sheet name based on the pay_key
            pay_data.to_excel(writer, index=False, sheet_name=pay_key, engine='xlsxwriter')
    # for pay_key, pay_data in payment_splits.items():
    #     file_name = f"{output_dir}{pay_key}.xlsx"  # Filename based on pay_key
    #     print(f"Saving {len(pay_data)} rows to file: {file_name}")
        
    #     # Save the split data to an Excel file
    #     pay_data.to_excel(file_name, index=False, engine='xlsxwriter')



In [ ]:
file_name = 'payment_splits.xlsx' 
save_payment_splits_to_excel(payment_final, file_name,5)

pay_data 690827


In [239]:
file_name = payment_path[0].split('\\')[1]
file_name

'payment_splits.xlsx'

In [ ]:
payment.sum()

In [ ]:
len(payment)